In [22]:
import pandas as pd
import pandas_datareader.data as web
from datetime import datetime, timedelta
import sqlite3

In [235]:
conn = sqlite3.connect("us_stocks.db")
c = conn.cursor()

In [241]:
max_date_available = list(c.execute("SELECT MAX(Date) FROM stocks_final"))[0][0]
max_date_available = datetime.strptime(max_date_available, "%Y-%m-%d %H:%M:%S" )

In [242]:
first_date_to_get = max_date_available + timedelta(1)

In [244]:
today = datetime.today() 
today = datetime(today.year, today.month, today.day, 0, 0)

In [245]:
business_dates = list(pd.bdate_range(first_date_to_get,today))

In [246]:
## If today is not equal to first date we want to get, we can run the code to produce range

if today != first_date_to_get:
    business_dates = list(pd.bdate_range(first_date_to_get,today))
    
    ## If today's date is in the list we have to drop it. It may happen when there is a weekend in between: 
    
    if today.date() in [ts.date() for ts in business_dates]:
        
    ## Dropping the last element
        business_dates.pop()
    
    ## Now we have a final list with business days for which data should be gathered
    
    ## Loading ticker, market pairs to iterate through
    ticker_market_list = list(c.execute("SELECT DISTINCT Ticker, Market FROM stocks_final"))
    
    ## Gathering data and saving to sql table
    
    for ticker, market in ticker_market_list:
        if len(business_dates) == 1:
    
        df = web.DataReader(ticker, 'yahoo', first_date_to_get, first_date_to_get)
        df["Ticker"] = ticker
        df["Market"] = market
        df.reset_index(inplace = True)
        df = df.iloc[:,[7,0,1,2,3,4,6,5,8]]
        df = df.drop_duplicates("Date")
        df.to_sql("stocks_final", conn, if_exists = 'append', index = False)

    else:

        last_date_to_get = business_dates[-1]

        df = web.DataReader(ticker, 'yahoo', first_date_to_get, last_date_to_get)
        df["Ticker"] = ticker
        df["Market"] = market
        df.reset_index(inplace = True)
        df = df.iloc[:,[7,0,1,2,3,4,6,5,8]]
        df.to_sql("stocks", conn, if_exists = 'append', index = False)